# ENGR 240: Exploring the Lorenz Equations

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%209/Worksheet%209-1%20Exploring%20Lorenz%20Equations.ipynb)

## Introduction

The Lorenz equations are a famous system of ODEs that exhibit chaotic behavior - small changes in initial conditions lead to dramatically different outcomes (the "butterfly effect"). Originally derived as a simplified model of atmospheric convection, these equations have become one of the most studied examples in chaos theory.

### The Lorenz System
$$\frac{dx}{dt} = \sigma(y - x)$$
$$\frac{dy}{dt} = rx - y - xz$$
$$\frac{dz}{dt} = xy - bz$$

**What do the variables represent?**

In the original atmospheric convection model:
- **x**: Rate of convective overturning (proportional to the intensity of fluid circulation)
- **y**: Horizontal temperature variation (temperature difference between rising and descending fluid)
- **z**: Vertical temperature variation (deviation from a linear temperature profile)

**Parameters:**
- **σ (sigma)**: Prandtl number - ratio of momentum diffusivity to thermal diffusivity
- **r**: Rayleigh number - measures the strength of convection driving forces
- **b**: Geometric factor related to the aspect ratio of convection cells

While derived from fluid dynamics, the Lorenz equations now serve as a canonical example of chaotic behavior in many fields including engineering, physics, biology, and economics.

**Learning Objectives:**
- Solve systems of ODEs using `scipy.integrate.solve_ivp`
- Explore how parameter changes affect system behavior
- Demonstrate sensitivity to initial conditions (chaos theory)
- Create 3D phase portraits

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 8]
print("Libraries imported successfully!")

## Task 1: Implement the Lorenz System

In [ ]:
def lorenz_system(t, state, sigma, r, b):
    """
    The Lorenz equations.
    state = [x, y, z], returns [dx/dt, dy/dt, dz/dt]
    """
    x, y, z = state
    
    dxdt = sigma * (y - x)
    dydt = r * x - y - x * z
    dzdt = x * y - b * z
    
    return [dxdt, dydt, dzdt]

# Test the function
test_derivs = lorenz_system(0, [1, 1, 1], 10, 28, 8/3)
print(f"Test derivatives: {test_derivs}")

## Task 2: Explore Different Parameter Regimes

Let's see how behavior changes with the parameter $r$ (Rayleigh number):
- **r = 1**: Stable fixed point (no convection, heat conducted)
- **r = 15**: Steady convection (periodic oscillation)  
- **r = 23**: Complex periodic behavior (multiple frequencies)
- **r = 28**: Chaotic convection (strange attractor)

In [ ]:
# Parameters
sigma, b = 10.0, 8.0/3.0
r_values = [1, 15, 23, 28]
initial_state = [2.0, 2.0, 2.0]
t_span = (0, 30)
t_eval = np.linspace(0, 30, 3000)

# Solve for each r value
solutions = {}
for r in r_values:
    print(f"Solving for r = {r}...")
    sol = solve_ivp(lorenz_system, t_span, initial_state,
                   args=(sigma, r, b), t_eval=t_eval, method='RK45')
    solutions[r] = {'t': sol.t, 'x': sol.y[0], 'y': sol.y[1], 'z': sol.y[2]}

print("All solutions computed!")

In [ ]:
# Plot time series and 3D phase portraits
fig = plt.figure(figsize=(16, 12))

for i, r in enumerate(r_values):
    sol = solutions[r]
    
    # Time series (top row)
    plt.subplot(2, 4, i+1)
    plt.plot(sol['t'], sol['x'], 'b-', linewidth=0.8)
    plt.title(f'r = {r}: x(t)')
    plt.xlabel('Time')
    plt.ylabel('x (convection rate)')
    plt.grid(True, alpha=0.3)
    
    # 3D phase portrait (bottom row)
    ax = fig.add_subplot(2, 4, i+5, projection='3d')
    ax.plot(sol['x'], sol['y'], sol['z'], 'r-', linewidth=0.5, alpha=0.8)
    ax.set_title(f'r = {r}: Phase Portrait')
    ax.set_xlabel('x (convection)')
    ax.set_ylabel('y (horizontal temp)')
    ax.set_zlabel('z (vertical temp)')

plt.tight_layout()
plt.show()

## Task 3: Demonstrate the Butterfly Effect

Small changes in initial conditions → dramatically different outcomes! This demonstrates why long-term weather prediction is fundamentally limited.

In [ ]:
# Three very similar initial conditions (chaotic case r = 28)
initial_conditions = [
    [2.000, 2.0, 2.0],  # Original
    [1.999, 2.0, 2.0],  # Tiny decrease
    [2.001, 2.0, 2.0]   # Tiny increase  
]

labels = ['x₀ = 2.000', 'x₀ = 1.999', 'x₀ = 2.001']
colors = ['blue', 'red', 'green']

# Solve for each initial condition
chaos_solutions = []
for i, ic in enumerate(initial_conditions):
    sol = solve_ivp(lorenz_system, t_span, ic, 
                   args=(sigma, 28, b), t_eval=t_eval, method='RK45', rtol=1e-10)
    chaos_solutions.append({
        't': sol.t, 'x': sol.y[0], 'y': sol.y[1], 'z': sol.y[2],
        'label': labels[i], 'color': colors[i]
    })

print("Chaos analysis complete!")

In [ ]:
# Compare the diverging solutions
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Plot x(t) for all three initial conditions
for sol in chaos_solutions:
    ax1.plot(sol['t'], sol['x'], color=sol['color'], 
             label=sol['label'], linewidth=1.5)

ax1.set_xlabel('Time')
ax1.set_ylabel('x (convection rate)')
ax1.set_title('Butterfly Effect: Convection Rate x(t) for r = 28')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Show exponential divergence
ref_x = chaos_solutions[0]['x']
for i in range(1, len(chaos_solutions)):
    diff = np.abs(chaos_solutions[i]['x'] - ref_x)
    ax2.semilogy(chaos_solutions[i]['t'], diff, 
                 color=chaos_solutions[i]['color'],
                 label=f"|x - x_ref| for {chaos_solutions[i]['label']}",
                 linewidth=1.5)

ax2.set_xlabel('Time')
ax2.set_ylabel('|Δx| (log scale)')
ax2.set_title('Exponential Divergence (Chaos!)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final differences
for i in range(1, len(chaos_solutions)):
    final_diff = abs(chaos_solutions[i]['x'][-1] - ref_x[-1])
    print(f"Final difference for {chaos_solutions[i]['label']}: {final_diff:.3f}")
    
print("\nThis demonstrates why weather prediction beyond ~2 weeks is fundamentally impossible!")

## Task 4: Student Exploration

**Your turn!** Complete the code below to explore intermediate r values and see the transition to chaos.

In [ ]:
# TODO: Explore intermediate r values between 23 and 28
explore_r_values = [24, 25, 26, 27]  # Try these values

# TODO: Create a figure with subplots for each r value
plt.figure(figsize=(16, 8))

for i, r in enumerate(explore_r_values):
    # TODO: Solve the Lorenz system for this r value
    # Hint: Use solve_ivp with the same parameters as before
    
    # sol = solve_ivp(...)
    
    # TODO: Plot x(t) in subplot
    plt.subplot(2, 2, i+1)
    # plt.plot(...)
    plt.title(f'r = {r}')
    plt.xlabel('Time')
    plt.ylabel('x (convection rate)')
    plt.grid(True)

plt.tight_layout()
plt.show()

print("Complete the TODO sections above to see the transition to chaos!")

## Task 5: Create Your Own Chaos

Experiment with different parameters and initial conditions!

In [ ]:
# TODO: Try different sigma values (try 5, 15, 20)
custom_sigma = 10  # Change this!
custom_r = 28
custom_b = 8/3

# TODO: Try different initial conditions
custom_initial = [1.0, 0.0, 0.0]  # Change this!

# TODO: Solve and plot your custom system
sol_custom = solve_ivp(lorenz_system, (0, 50), custom_initial,
                      args=(custom_sigma, custom_r, custom_b), 
                      t_eval=np.linspace(0, 50, 5000), method='RK45')

# Create 3D plot
fig = plt.figure(figsize=(12, 5))

# Time series
plt.subplot(1, 2, 1)
plt.plot(sol_custom.t, sol_custom.y[0], 'b-', linewidth=0.8)
plt.title(f'Custom System: σ={custom_sigma}, r={custom_r}')
plt.xlabel('Time')
plt.ylabel('x (convection rate)')
plt.grid(True)

# 3D phase portrait
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot(sol_custom.y[0], sol_custom.y[1], sol_custom.y[2], 'r-', linewidth=0.5)
ax.set_title('3D Phase Portrait')
ax.set_xlabel('x (convection)')
ax.set_ylabel('y (horizontal temp)')
ax.set_zlabel('z (vertical temp)')

plt.tight_layout()
plt.show()

## Discussion Questions

1. **How does the convection behavior change as r increases from 1 to 28?**

2. **What does the butterfly effect tell us about weather prediction?**

3. **Why is the r = 28 case called a "strange attractor"?**

4. **What numerical challenges might arise when solving chaotic systems?**

## Key Takeaways

- **Chaos**: Deterministic systems can exhibit unpredictable behavior
- **Sensitivity**: Tiny changes → huge differences (butterfly effect)
- **Attractors**: Long-term behavior confined to specific regions
- **Numerical methods**: `solve_ivp` handles complex ODE systems effectively
- **Applications**: Weather, populations, economics, engineering systems

The Lorenz equations demonstrate that even simple mathematical models can exhibit incredibly complex behavior - a cornerstone of chaos theory that has profound implications for understanding natural systems!